In [30]:
!pip install seaborn
!pip install xgboost
!pip install --upgrade xgboost
!pip install catboost
!pip install pycaret #pycaret 설치
!pip install category_encoders
!pip install wandb


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import os
import json
import pandas as pd
from pycaret.classification import *
from preprocess import DataPreprocessor
import wandb
from sklearn.preprocessing import MinMaxScaler

In [2]:
wandb.login()
wandb.init(project='automl')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ans2568 (ai-leaders). Use `wandb login --relogin` to force relogin


In [2]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [3]:
df_test = df_test.drop(['id'], axis=1)

In [4]:
with open(os.path.join('config', 'config.json')) as f:
    preprocess_columns = json.load(f)

In [5]:
preprocessor = DataPreprocessor(preprocess_columns)
df_train_processed = preprocessor.fit_transform(df_train)
df_test_processed = preprocessor.transform(df_test)

In [7]:
import numpy as np
def outlier_iqr(data, column): 

    # lower, upper 글로벌 변수 선언하기     
    global lower, upper    
    
    # 4분위수 기준 지정하기     
    q25, q75 = np.quantile(data[column], 0.25), np.quantile(data[column], 0.75)          
    
    # IQR 계산하기     
    iqr = q75 - q25    
    
    # outlier cutoff 계산하기     
    cut_off = iqr * 1.5          
    
    # lower와 upper bound 값 구하기     
    lower, upper = q25 - cut_off, q75 + cut_off     
    
    print('IQR은',iqr, '이다.')     
    print('lower bound 값은', lower, '이다.')     
    print('upper bound 값은', upper, '이다.')    
    
    # 1사 분위와 4사 분위에 속해있는 데이터 각각 저장하기     
    data1 = data[data[column] > upper]     
    data2 = data[data[column] < lower]    
    
    # 이상치 총 개수 구하기
    return print('총 이상치 개수는', data1.shape[0] + data2.shape[0], '이다.'), lower, upper

In [8]:
def removeOutlier(data, column):
    _, lower, upper = outlier_iqr(data, column)
    data = data[(data[column] > lower) & (data[column] < upper)]
    return data

In [9]:
df_train_processed = removeOutlier(df_train_processed, 'customer_country')
df_train_processed = removeOutlier(df_train_processed, 'customer_position')

IQR은 21.0 이다.
lower bound 값은 -29.5 이다.
upper bound 값은 54.5 이다.
총 이상치 개수는 5522 이다.
IQR은 4.0 이다.
lower bound 값은 -2.0 이다.
upper bound 값은 14.0 이다.
총 이상치 개수는 1003 이다.


In [10]:
df_train_processed

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_type,enterprise_1,enterprise_2,historical_existing_cnt,idit_strategic_ver,customer_job,...,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,lead_owner,is_converted
0,1.00,1,1,0.066667,1,1,0,0.0,0.0,1,...,1,1,1,1,0,0.003079,0.026846,1,0.007235,True
1,1.00,1,1,0.066667,1,1,0,12.0,0.0,2,...,2,1,1,1,0,0.003079,0.026846,1,0.000337,True
2,1.00,2,1,0.088889,1,1,0,144.0,0.0,3,...,3,2,1,1,0,0.003079,0.026846,1,0.000675,True
3,1.00,2,1,0.088889,1,1,0,0.0,0.0,4,...,2,2,1,1,0,0.003079,0.026846,1,0.000489,True
4,1.00,2,1,0.088889,2,1,0,0.0,0.0,5,...,3,2,1,0,0,0.003079,0.026846,1,0.001282,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59293,1.00,50,1,0.000000,7,0,1,0.0,0.0,3,...,1,28,3,0,0,0.000026,0.028777,13,0.003592,False
59294,1.00,50,1,0.000000,7,0,1,0.0,0.0,3,...,1,28,3,0,0,0.000026,0.028777,13,0.003592,False
59295,0.75,10,1,0.040000,2,1,0,0.0,0.0,4,...,11,10,4,0,0,0.000026,0.028777,13,0.005127,False
59296,0.75,42,1,0.040000,2,1,0,0.0,0.0,3,...,1,24,1,0,0,0.000026,0.028777,13,0.011636,False


In [12]:
scale_col = ["historical_existing_cnt", "lead_desc_length", "customer_idx", "customer_country", 
        "business_unit",
        "customer_type",
        "customer_job",
        "inquiry_type",
        "product_category",
        "customer_position",
        "response_corporate",
        "expected_timeline",
        "business_area",
        "product_modelname" ]
scaler = MinMaxScaler()
train_len = len(df_train_processed)
df_all = pd.concat([df_train_processed, df_test_processed])
df_all[scale_col] = scaler.fit_transform(df_all[scale_col])
df_train_processed = df_all[:train_len]
df_test_processed = df_all[train_len:]

In [13]:
df_train_processed

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_type,enterprise_Enterprise,enterprise_SMB,historical_existing_cnt,idit_strategic_ver,customer_job,...,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,lead_owner,is_converted
0,1.00,0.005495,0.0,0.066667,0.00,1,0,0.000000,0.0,0.010753,...,0.000000,0.00,0.000000,1,0,0.003079,0.026846,0.0,0.007235,True
1,1.00,0.005495,0.0,0.066667,0.00,1,0,0.005408,0.0,0.016129,...,0.018182,0.00,0.000000,1,0,0.003079,0.026846,0.0,0.000337,True
2,1.00,0.008242,0.0,0.088889,0.00,1,0,0.064894,0.0,0.021505,...,0.036364,0.02,0.000000,1,0,0.003079,0.026846,0.0,0.000675,True
3,1.00,0.008242,0.0,0.088889,0.00,1,0,0.000000,0.0,0.026882,...,0.018182,0.02,0.000000,1,0,0.003079,0.026846,0.0,0.000489,True
4,1.00,0.008242,0.0,0.088889,0.04,1,0,0.000000,0.0,0.032258,...,0.036364,0.02,0.000000,0,0,0.003079,0.026846,0.0,0.001282,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59293,1.00,0.140110,0.0,0.000000,0.24,0,1,0.000000,0.0,0.021505,...,0.000000,0.54,0.036364,0,0,0.000026,0.028777,1.0,0.003592,False
59294,1.00,0.140110,0.0,0.000000,0.24,0,1,0.000000,0.0,0.021505,...,0.000000,0.54,0.036364,0,0,0.000026,0.028777,1.0,0.003592,False
59295,0.75,0.030220,0.0,0.040000,0.04,1,0,0.000000,0.0,0.026882,...,0.181818,0.18,0.054545,0,0,0.000026,0.028777,1.0,0.005127,False
59296,0.75,0.118132,0.0,0.040000,0.04,1,0,0.000000,0.0,0.021505,...,0.000000,0.46,0.000000,0,0,0.000026,0.028777,1.0,0.011636,False


In [14]:
df_test_processed

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_type,enterprise_Enterprise,enterprise_SMB,historical_existing_cnt,idit_strategic_ver,customer_job,...,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,lead_owner,is_converted
0,0.00,0.491758,0.25,0.073248,0.24,1,0,0.023885,0.0,0.032258,...,0.127273,0.10,0.018182,1,0,0.001183,0.049840,0.666667,0.001568,False
1,0.25,0.282967,0.50,0.000000,0.24,0,1,0.000000,0.0,0.048387,...,0.127273,0.16,0.018182,0,0,0.000013,0.000000,0.916667,0.002378,True
2,1.00,0.494505,0.25,0.000000,0.04,0,1,0.000000,0.0,0.069892,...,0.054545,0.12,0.000000,0,0,0.000060,0.131148,0.333333,0.000000,True
3,0.50,0.282967,0.25,0.118644,0.16,1,0,0.000000,0.0,0.043011,...,0.127273,0.16,0.072727,0,0,0.001183,0.049840,0.666667,0.037235,False
4,1.00,0.491758,0.25,0.074949,0.24,1,0,0.000901,1.0,0.021505,...,0.127273,0.10,0.000000,1,1,0.003079,0.064566,0.000000,0.007740,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,0.50,0.019231,0.00,0.000000,0.16,1,0,0.004507,0.0,0.043011,...,0.090909,0.10,0.018182,0,0,0.000000,0.000000,0.083333,0.042142,False
5267,0.25,0.282967,0.50,0.000000,0.16,1,0,0.000000,0.0,0.048387,...,0.127273,0.16,0.018182,0,0,0.000000,0.000000,0.083333,0.000472,True
5268,0.75,0.491758,0.00,0.000000,0.04,1,0,0.000000,0.0,0.026882,...,0.054545,0.10,0.000000,0,0,0.000000,0.000000,0.083333,0.042142,False
5269,0.00,0.510989,0.50,0.000000,0.24,0,1,0.000000,0.0,0.048387,...,0.127273,0.48,0.018182,0,0,0.000000,0.000000,0.083333,0.000236,True


In [15]:
model = setup(
    data = df_train_processed,
    train_size=0.8,
    target = "is_converted",
    fold = 10,
    n_jobs=1,
    log_experiment='wandb',
)

,Description,Value
0,Session id,2717
1,Target,is_converted
2,Target type,Binary
3,Original data shape,"(52771, 24)"
4,Transformed data shape,"(52771, 24)"
5,Transformed train set shape,"(42216, 24)"
6,Transformed test set shape,"(10555, 24)"
7,Numeric features,23
8,Preprocess,True
9,Imputation type,simple


In [16]:
xgboost = create_model('xgboost')
final_model = finalize_model(xgboost)

In [17]:
# 예측에 필요한 데이터 분리
x_test = df_test_processed.drop(["is_converted"], axis=1)

pred = predict_model(final_model, data=x_test)

In [18]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = pred["prediction_label"].astype(bool)
df_sub

,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,53.0,NaN,...,LGESP,NaN,1,0,0.001183,0.049840,retail,Electronics & Telco,278,False
1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,NaN,5405,End Customer,SMB,NaN,NaN,...,LGEUS,NaN,0,0,0.000013,NaN,transportation,Others,437,True
2,8491,1.00,/ / U.A.E,ID,NaN,13597,Specifier/ Influencer,SMB,NaN,NaN,...,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,False
3,19895,0.50,/ Madison / United States,ID,0.118644,17204,NaN,Enterprise,NaN,NaN,...,LGEUS,more than a year,0,0,0.001183,0.049840,retail,NaN,194,False
4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,2.0,1.0,...,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,167,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,13855,0.50,/São Paulo/Brazil,AS,NaN,40292,NaN,Enterprise,10.0,NaN,...,LGESP,NaN,0,0,NaN,NaN,NaN,NaN,97,False
5267,7979,0.25,General / / United States,IT,NaN,47466,NaN,Enterprise,0.0,NaN,...,LGEUS,NaN,0,0,NaN,NaN,NaN,NaN,438,False
5268,12887,0.75,/ OURO BRANCO / Brazil,AS,NaN,46227,Specifier/ Influencer,Enterprise,NaN,NaN,...,LGESP,less than 3 months,0,0,NaN,NaN,NaN,NaN,97,False
5269,17530,0.00,/ / Germany,IT,NaN,45667,End Customer,SMB,NaN,NaN,...,LGEDG,NaN,0,0,NaN,NaN,NaN,NaN,429,False


In [20]:
# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)